In [5]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [6]:
from sklearn.pipeline import make_pipeline

In [7]:
# ! pip install pandas
# ! pip install mlflow

In [8]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("chapter-4")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1718521577410, experiment_id='1', last_update_time=1718521577410, lifecycle_stage='active', name='chapter-4', tags={}>

In [9]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [10]:
df_train = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment-tracking/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/02-experiment-tracking/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [11]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/home/codespace/.local/share/virtualenvs/web-service-ToH4o1-o/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
from mlflow.tracking import MlflowClient


In [15]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '3c55362ba332448e95ccf981547f7eaa'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv

DictVectorizer()